In [1]:
import torch

# Check if MPS is available
print(torch.backends.mps.is_available())

True


In [5]:
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 225.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 5.2 MB/s eta 0:00:0000:010:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 5.1 MB/s eta 0:00:0000:0100:01


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# 检查MPS是否可用并设置设备
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [4]:
device

device(type='mps')

In [5]:
# 加载数据集
# 尝试使用不同的编码读取文件
encodings = ['utf-8','GBK']

for enc in encodings:
    try:
        data = pd.read_csv('dataset.csv', encoding=enc)
        print(f"Successfully read the file with encoding: {enc}")
        break
    except UnicodeDecodeError:
        print(f"Failed to decode with encoding: {enc}")
        continue

Failed to decode with encoding: utf-8
Successfully read the file with encoding: GBK


In [6]:
#文本和标签
texts = data['text'].tolist()
labels = data['label'].tolist()

# 创建标签到数字的映射
unique_labels = list(set(labels))
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for idx, label in enumerate(unique_labels)}

# 将文本标签转换为数字ID
numeric_labels = [label2id[label] for label in labels]

# 分割数据集
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, numeric_labels, test_size=0.1)


In [7]:
texts[0]


'【自营】赖氨酸片生长素青少年学生成年人补钙片黄金乳钙骨骼长高'

In [8]:
# 加载预训练的BERT模型和分词器
model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=248).to(device)

/opt/homebrew/anaconda3/envs/py3.9/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [11]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# 检查MPS设备
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# 加载中文BERT分类模型
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=len(unique_labels))
model.to(device)

# 训练参数设置
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 训练的轮数
    per_device_train_batch_size=16,  # 训练时每个设备的批量大小
    per_device_eval_batch_size=16,   # 评估时每个设备的批量大小
    warmup_steps=500,                # 学习率预热步数
    weight_decay=0.01,               # 权重衰减
    logging_dir='./logs',            # 日志目录
)

trainer = Trainer(
    model=model,                         # 训练的模型
    args=training_args,                  # 训练参数
    train_dataset=train_dataset,         # 训练数据集
    eval_dataset=val_dataset             # 评估数据集
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# 训练模型
trainer.train()

Step,Training Loss
500,4.592900


TrainOutput(global_step=684, training_loss=4.07754253365143, metrics={'train_runtime': 316.0485, 'train_samples_per_second': 34.514, 'train_steps_per_second': 2.164, 'total_flos': 719069297107968.0, 'train_loss': 4.07754253365143, 'epoch': 3.0})

In [13]:
# 保存训练好的模型
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')

In [24]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 加载模型和分词器
model = BertForSequenceClassification.from_pretrained('./model')
model.to(device)
tokenizer = BertTokenizer.from_pretrained('./model')

# 预测函数
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return id2label[predicted_class]

# 示例推理
sample_text = "口罩"
print(predict(sample_text))


医疗器械
